# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Domherren der Personendatenbank einlesen

kompakte Version 2022-11-22

[Vorspann](#Vorspann)  
[Literatur](#Literatur)  
[Item](#Item)  
[Person](#Person)  
[Externe ID](#Externe-ID)  
[Namensvarianten](#Namensvarianten)  
[Suchvarianten](#Suchvarianten)  

### Vorspann

In [1]:
wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\Georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
db = "wiag_in"

"wiag_in"

In [9]:
Wds.setDBWIAG(user="georg", db = db)

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

Evtl. neue Quellen eintragen!

In [10]:
gso_db = "gso_in_202211";

In [11]:
item_type_id = 6

6

In [12]:
ENV["COLUMNS"] = 120

120

In [13]:
using CSV, DataFrames, MySQL

In [14]:
includet("src/Util.jl")

In [15]:
includet("src/Domherr-GS.jl")

In [16]:
check_globals()

┌ Info: WiagDataSetup
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:13
┌ Info: 
│   item_type_id = 6
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:14
┌ Info: 
│   gso_db = gso_in_202211
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:15


### Tabellen löschen
Tabelle `item` muss zuletzt gelöscht werden! (ausgenommen: `reference_volume`, `person` mit `item_type_id`)

In [113]:
table = "item_property"
clear_item_property!(Wds.dbwiag, table, "item_id", "academic_title")

┌ Info: Zeilen 
│   n_row = 443
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:70


443

In [114]:
clear_table!(Wds.dbwiag, "id_external", "item_id")

┌ Info: Zeilen 
│   n_row = 10680
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


10680

In [115]:
clear_table!(Wds.dbwiag, "familyname_variant", "person_id")

┌ Info: Zeilen 
│   n_row = 2940
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


2940

In [116]:
clear_table!(Wds.dbwiag, "givenname_variant", "person_id")

┌ Info: Zeilen 
│   n_row = 1639
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


1639

In [117]:
clear_table!(Wds.dbwiag, "name_lookup", "person_id")

┌ Info: Zeilen 
│   n_row = 17301
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


17301

In [118]:
clear_table!(Wds.dbwiag, "person_role", "person_id")

┌ Info: Zeilen 
│   n_row = 16525
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


16525

In [119]:
clear_table!(Wds.dbwiag, "person", "id")

┌ Info: Zeilen 
│   n_row = 9344
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


9344

## Vorspann `gsn_gsn`

``` sql
use gso_in_202211;

CREATE TEMPORARY TABLE gsn_new AS
SELECT id, item_id, nummer
FROM gsn
WHERE id IN
(SELECT min(id) as id from gsn where not deleted group by item_id);

CREATE TABLE gsn_gsn AS
SELECT o.id, o.item_id, o.nummer, n.id as id_new, n.nummer as nummer_neu
FROM gsn AS o
INNER JOIN gsn_new AS n ON o.item_id = n.item_id
WHERE NOT o.deleted;
```

### Literatur

In [17]:
table = "reference_volume"
count_table(Wds.dbwiag, table, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM reference_volume WHERE item_type_id = 6
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


89

In [18]:
table = "reference_volume"
c = clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen 
│   n_row = 89
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


89

In [19]:
gso_table = "books"
insert_reference_volume!(table, gso_table)

┌ Info: reading 
│   table = reference_volume
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:25
┌ Info: Zeilen, Spalten: 
│   size(df) = (89, 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:35
┌ Info: Rows inserted: 89
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


89

Feld `gs_citation`

In [20]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data";
gs_citation_file = "GS_Baende_ZitationGS.csv";
source_file = joinpath(data_path, gs_citation_file);
update_gs_citation!(table, source_file)

93

*2022-11-22* Es gibt nur 89 Einträge für Referenzwerke in der Personendatenbank aber 93 Zitierweisen für GS?!

### Personendaten
Finde Einträge, die von der Domherrendatenbank referenziert werden (sofern die verweisenden Domherren online sind!)

In [95]:
db_ref = "wiag_in"
df_person_dh = person_referenced(db_ref);

┌ Info: check for ambiguous GSN
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:93
┌ Info: Zahl der eindeutigen GSN
│   size(df_gsn, 1) = 1163
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:104
┌ Info: Zahl der zugeordneten Einträge in `persons`
│   size(df_person, 1) = 1163
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:120


In [96]:
size(df_person_dh)

(1163, 3)

In [97]:
names(df_person_dh)

3-element Vector{String}:
 "dh_item_id"
 "gsn"
 "person_id"

Domherren GS: online, mit einem Amt in einem der Domstifte

In [99]:
db_ref = "wiag_in"
df_person_dft = person_by_domstift(db_ref);

┌ Info: Personen mit einem Amt in einem Domstift
│   size(df_p_dft, 1) = 9104
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:163


In [139]:
names(df_person_dft)

3-element Vector{String}:
 "person_id"
 "item_id"
 "gsn"

In [100]:
columns = [:gsn, :person_id]

2-element Vector{Symbol}:
 :gsn
 :person_id

vereinigen und jeden Eintrag nur einmal verwenden

In [101]:
df_person_u = vcat(select(df_person_dft, columns), select(df_person_dh, columns));

In [103]:
df_person = unique(df_person_u, :person_id);

In [104]:
size(df_person)

(9346, 2)

Personendaten einlesen

In [105]:
df_p_all = read_person();

In [106]:
size(df_p_all)

(81526, 16)

betrachte nur Domherren

In [107]:
df_p = innerjoin(df_p_all, df_person, on = :person_id);

In [109]:
size(df_p)

(9346, 17)

### Einträge in `item`

ordne `id_wiag` zu, bzw. vergib eine neue ID

In [110]:
db_ref = "wiag2"
df_p_id = set_id_wiag(df_p, db_ref);

┌ Info: Zahl der erzeugten IDs
│   new_n = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:268


`id_in_source` ist `person_id` = `persons.id` als `string`

In [88]:
maximum(df_p_id.id_wiag)

91366

In [120]:
table_name = "item";
count_table(Wds.dbwiag, table_name, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM item WHERE item_type_id = 6
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


9344

In [121]:
table_name = "item";
clear_table!(Wds.dbwiag, table_name, "item_type_id")

┌ Info: Zeilen 
│   n_row = 9344
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


9344

In [122]:
table_name = "item"
insert_item!(table_name, df_p_id);

┌ Info: Rows inserted: 9346
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


### Personendaten

In [123]:
table = "person"
count_table(Wds.dbwiag, table, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM person WHERE item_type_id = 6
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


0

In [124]:
table = "person"
clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen 
│   n_row = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


0

In [125]:
table = "person"
insert_person!(table, df_p_id)

┌ Info: unbekannte Ordensbezeichnungen
│   unique(unknown_order) = ["Kapuziner", "S. J."]
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:344
┌ Info: Rows inserted: 9346
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


9346

### Akademischer Titel

In [126]:
table = "item_property"
clear_item_property!(Wds.dbwiag, table, "item_id", "academic_title")

┌ Info: Zeilen 
│   n_row = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:70


0

In [127]:
table = "item_property"
insert_item_property!(table, df_p_id, property_column = Symbol("titel"), property_name = "academic_title")

┌ Info: Rows inserted: 443
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


443

### Referenzverweise

In [128]:
table = "item_reference"
count_table(Wds.dbwiag, table, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM item_reference WHERE item_type_id = 6
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


8202

In [129]:
table = "item_reference"
clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen 
│   n_row = 8202
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


8202

In [130]:
table = "item_reference"
df_ref = insert_item_reference!(table, df_p_id);

┌ Info: Zeilen, Spalten
│   size(df_idx) = (9346, 2)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:437
┌ Info: Zeilen, Spalten
│   size(df_ref) = (15630, 5)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:453
┌ Info: Zeilen, Spalten (mit Biogramm)
│   size(df_ref_bio) = (8204, 5)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:459
┌ Info: Rows inserted: 8204
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


### Externe ID

In [35]:
df_p_id[551:555, [:id, :item_id, :id_in_source, :id_wiag, :vorname, :familienname]]

Row,id,item_id,id_in_source,id_wiag,vorname,familienname
,Int32,Int32,String,Int64?,String?,String?
1,12309,12309,12309,80640,Kaspar,Amstedt
2,12312,12312,12312,80641,Kaspar,Han
3,12313,12313,12313,80642,Kaspar,Moer
4,12320,12320,12320,80643,Kaspar,Withop
5,12360,12360,12360,80645,Christian,Schulte


In [131]:
filter(names(df_p_id)) do col
    col in ("viaf", "gndnummer", "gsn")
end

3-element Vector{String}:
 "gndnummer"
 "viaf"
 "gsn"

In [132]:
table = "id_external"
count_table(Wds.dbwiag, table, "item_id")

┌ Info: SELECT COUNT(*) as n FROM id_external WHERE item_id IN (SELECT id FROM item WHERE item_type_id = 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


0

In [133]:
clear_table!(Wds.dbwiag, table, "item_id")

┌ Info: Zeilen 
│   n_row = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


0

In [134]:
table = "id_external"
insert_id_external!(table, df_p_id)

┌ Info: Normdatum 
│   auth.second = gndnummer
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:513
┌ Info: Rows inserted: 1334
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216
┌ Info: Normdatum 
│   auth.second = viaf
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:513
┌ Info: Rows inserted: 2
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216
┌ Info: Normdatum 
│   auth.second = gsn
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:513
┌ Info: Rows inserted: 9346
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


Es gibt keine Angaben zu Wikipedia-Einträgen.

### Namensvarianten

In [135]:
table = "familyname_variant"
col_fn_variant = :familiennamenvarianten

:familiennamenvarianten

In [136]:
count_table(Wds.dbwiag, table, "person_id")

┌ Info: SELECT COUNT(*) as n FROM familyname_variant WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


283

In [137]:
clear_table!(Wds.dbwiag, table, "person_id")

┌ Info: Zeilen 
│   n_row = 283
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


283

In [138]:
table = "familyname_variant";
insert_name_variant!(table, df_p, col_fn_variant)

┌ Info: Rows inserted: 2942
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


2942

In [139]:
table = "givenname_variant"
col_gn_variant = :vornamenvarianten

:vornamenvarianten

In [140]:
count_table(Wds.dbwiag, table, "person_id")

┌ Info: SELECT COUNT(*) as n FROM givenname_variant WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


572

In [141]:
clear_table!(Wds.dbwiag, table, "person_id")

┌ Info: Zeilen 
│   n_row = 572
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


572

In [142]:
table = "givenname_variant"
insert_name_variant!(table, df_p, col_gn_variant)

┌ Info: Rows inserted: 1639
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


1639

### Suchvarianten

In [143]:
table = "name_lookup"

"name_lookup"

In [144]:
count_table(Wds.dbwiag, table, "person_id")

┌ Info: SELECT COUNT(*) as n FROM name_lookup WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


3

In [145]:
table = "name_lookup"
clear_table!(Wds.dbwiag, table, "person_id")

┌ Info: Zeilen 
│   n_row = 3
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:46


3

In [146]:
table = "name_lookup"
insert_name_lookup!(table, df_p_id);

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: Rows inserted: 17305
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


### Bezug zu anderen Quellen

2022-12-07 Weiterentwicklung Bezug zu anderen Quellen; gemeinsame Verweise auf gs einmal von ep, einmal von dh

In [147]:
table = "canon_lookup"
df = map_sources(table);

┌ Info: Domherren mit Veweisen auf Bischöfe
│   size(df) = (419, 4)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:885
┌ Info: Indirekte Verweise auf Bischöfe durch gemeinsame Verweise auf Domherren GS
│   df_gs_double = 8×3 DataFrame
 Row │ ep_id   gs_id   dh_id  
     │ Int32   Int32   Int32  
─────┼────────────────────────
   1 │ 560543  693200  639409
   2 │ 562252  693257  642050
   3 │ 562262  695293  636727
   4 │ 560271  696932  660258
   5 │ 560266  696966  638221
   6 │ 562590  697274  658563
   7 │ 560631  698289  637588
   8 │ 560608  699030  651498
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:937
┌ Info: Verweise von Domherren auf Domherren-GS
│   size(df_dh_gs) = (832, 2)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS.jl:950
┌ Info: Restlice Bischöfe mit Verweis auf Domherren GS
│   size(df_ep_gs) = (455, 2)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Domherr-GS